## Supervised Machine Learning - Random Forest Classifier Algoritm (Star vs. QSO)

### Dependencies:

In [1]:
import numpy as np
import pandas as pd

from pathlib import Path
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.metrics import classification_report_imbalanced
from imblearn.ensemble import EasyEnsembleClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression


import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from sqlalchemy import extract

In [2]:
columns = ["obj_ID",'alpha','delta', 'u', 'g', 'r', 'i', 'z', 'run_ID',
       'rerun_ID', 'cam_col', 'field_ID', 'spec_obj_ID', 'redshift',
       'plate', 'MJD', 'fiber_ID']
target = ["class"]

In [3]:
# # Create engine
# engine = create_engine("sqlite:///

In [4]:
# # reflect an existing database into a new model
# Base = automap_base()
# # reflect the tables
# Base.prepare(engine, reflect=True)
# # Save references to each table
# Measurement = Base.classes.measurement
# Station = Base.classes.station

In [5]:
# # Create our session (link) from Python to the DB
# session = Session(engine)

In [6]:
# Load the data
file_path = Path('Resources/star_qso.csv', index=False)
data_df = pd.read_csv(file_path)
# df = df.loc[:, columns].copy()
data_df

,obj_ID,alpha,delta,u,g,r,i,z,run_ID,rerun_ID,cam_col,field_ID,spec_obj_ID,class,redshift,plate,MJD,fiber_ID
0,1.237680e+18,340.995120,20.589476,23.48827,23.33776,21.32195,20.25615,19.54544,8102,301,3,110,5.658980e+18,QSO,1.424659,5026,55855,741
1,1.237680e+18,23.234926,11.418188,21.46973,21.17624,20.92829,20.60826,20.42573,7773,301,2,462,1.250000e+19,QSO,0.586455,11069,58456,113
2,1.237670e+18,39.149691,28.102842,21.74669,20.03493,19.17553,18.81823,18.65422,5934,301,4,122,2.751760e+18,STAR,-0.000008,2444,54082,232
3,1.237680e+18,345.801874,32.672868,23.17274,20.14496,19.41948,19.22034,18.89359,8157,301,2,38,7.323010e+18,STAR,0.000072,6504,56540,574
4,1.237680e+18,353.201522,3.080796,24.54890,21.44267,20.95315,20.79360,20.48442,7712,301,5,284,4.822280e+18,STAR,-0.000429,4283,55864,178
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26221,1.237680e+18,353.951560,19.899651,21.61376,21.72610,21.39746,21.15175,21.44469,8096,301,4,190,8.559290e+18,QSO,1.954003,7602,56954,716
26222,1.237660e+18,128.395984,25.467238,19.77835,19.37797,19.28145,19.12017,19.05171,4335,301,5,123,2.170890e+18,QSO,0.465272,1928,53327,562
26223,1.237660e+18,247.224915,38.037879,21.81185,21.44972,21.19963,21.36909,20.80581,3225,301,2,246,1.207990e+19,QSO,0.943422,10729,58248,324
26224,1.237660e+18,136.329001,6.426314,18.96920,18.76108,18.46569,18.48491,18.35552,3031,301,3,109,1.342220e+18,QSO,1.029967,1192,52649,530


In [7]:
data_df.set_index(['spec_obj_ID'], inplace = True)

In [8]:
data_df

,obj_ID,alpha,delta,u,g,r,i,z,run_ID,rerun_ID,cam_col,field_ID,class,redshift,plate,MJD,fiber_ID
spec_obj_ID,,,,,,,,,,,,,,,,,
5.658980e+18,1.237680e+18,340.995120,20.589476,23.48827,23.33776,21.32195,20.25615,19.54544,8102,301,3,110,QSO,1.424659,5026,55855,741
1.250000e+19,1.237680e+18,23.234926,11.418188,21.46973,21.17624,20.92829,20.60826,20.42573,7773,301,2,462,QSO,0.586455,11069,58456,113
2.751760e+18,1.237670e+18,39.149691,28.102842,21.74669,20.03493,19.17553,18.81823,18.65422,5934,301,4,122,STAR,-0.000008,2444,54082,232
7.323010e+18,1.237680e+18,345.801874,32.672868,23.17274,20.14496,19.41948,19.22034,18.89359,8157,301,2,38,STAR,0.000072,6504,56540,574
4.822280e+18,1.237680e+18,353.201522,3.080796,24.54890,21.44267,20.95315,20.79360,20.48442,7712,301,5,284,STAR,-0.000429,4283,55864,178
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8.559290e+18,1.237680e+18,353.951560,19.899651,21.61376,21.72610,21.39746,21.15175,21.44469,8096,301,4,190,QSO,1.954003,7602,56954,716
2.170890e+18,1.237660e+18,128.395984,25.467238,19.77835,19.37797,19.28145,19.12017,19.05171,4335,301,5,123,QSO,0.465272,1928,53327,562
1.207990e+19,1.237660e+18,247.224915,38.037879,21.81185,21.44972,21.19963,21.36909,20.80581,3225,301,2,246,QSO,0.943422,10729,58248,324


In [9]:
#drop columns not needed for machine learning 
new_data_df=data_df.drop(['alpha','delta','run_ID','rerun_ID', 'cam_col', 'field_ID', "obj_ID",'plate', 'MJD', 'fiber_ID'], axis = 1 )

In [10]:
new_data_df

,u,g,r,i,z,class,redshift
spec_obj_ID,,,,,,,
5.658980e+18,23.48827,23.33776,21.32195,20.25615,19.54544,QSO,1.424659
1.250000e+19,21.46973,21.17624,20.92829,20.60826,20.42573,QSO,0.586455
2.751760e+18,21.74669,20.03493,19.17553,18.81823,18.65422,STAR,-0.000008
7.323010e+18,23.17274,20.14496,19.41948,19.22034,18.89359,STAR,0.000072
4.822280e+18,24.54890,21.44267,20.95315,20.79360,20.48442,STAR,-0.000429
...,...,...,...,...,...,...,...
8.559290e+18,21.61376,21.72610,21.39746,21.15175,21.44469,QSO,1.954003
2.170890e+18,19.77835,19.37797,19.28145,19.12017,19.05171,QSO,0.465272
1.207990e+19,21.81185,21.44972,21.19963,21.36909,20.80581,QSO,0.943422


### Variable engineering

In [11]:
# bin by quartile

frequency = [1, 2, 3]

u_size_bins = [-10000, -6600,-3310, 33]
new_data_df["binned_u"] = pd.cut(new_data_df["u"], u_size_bins, labels = frequency)

g_size_bins = [-10000, -6600,-3310, 33]
new_data_df["binned_g"] = pd.cut(new_data_df["g"], g_size_bins, labels = frequency)

r_size_bins = [9,16,23, 30]
new_data_df["binned_r"] = pd.cut(new_data_df["r"], r_size_bins, labels = frequency)

i_size_bins = [9,17,25, 33]
new_data_df["binned_i"] = pd.cut(new_data_df["i"], i_size_bins, labels = frequency)

z_size_bins = [-10000, -6600,-3310, 30]
new_data_df["binned_z"] = pd.cut(new_data_df["z"], z_size_bins, labels = frequency)

new_data_df.sample(30)

,u,g,r,i,z,class,redshift,binned_u,binned_g,binned_r,binned_i,binned_z
spec_obj_ID,,,,,,,,,,,,
1.032020e+19,20.68903,20.57674,20.52822,20.13095,19.90171,QSO,1.890345,3,3,2,2,3
3.233750e+18,20.66188,19.84179,19.50397,19.45508,19.36227,STAR,0.000803,3,3,2,2,3
4.123050e+18,22.01626,21.61475,21.53734,21.47629,20.73699,QSO,2.071578,3,3,2,2,3
7.398120e+17,19.29061,19.08761,18.90418,18.83548,18.81557,QSO,1.224391,3,3,2,2,3
5.851490e+18,23.30882,21.02787,20.55792,20.35111,20.14070,STAR,-0.000682,3,3,2,2,3
6.924550e+18,21.38173,20.53747,20.36608,20.04316,19.54247,QSO,0.790081,3,3,2,2,3
3.704330e+18,18.99455,18.04598,17.63787,17.44783,17.38555,STAR,-0.000624,3,3,2,2,3
4.544380e+18,22.58356,21.87566,21.74103,21.86254,21.66203,STAR,0.000219,3,3,2,2,3
2.609960e+18,21.76279,20.72943,20.15505,19.92506,19.64538,STAR,-0.000169,3,3,2,2,3


In [12]:
new_data_df.iloc[new_data_df.values==np.inf]
new_data_df =new_data_df[~new_data_df.isin([np.nan, np.inf, -np.inf]).any(1)]

### Test and Train Data

In [13]:
# Create our features
X = new_data_df.drop('class', axis=1)

# Create our target
y = new_data_df['class']

Counter(y)

Counter({'QSO': 12137, 'STAR': 14089})

In [14]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1,stratify=y)
Counter(y_train)

Counter({'QSO': 9103, 'STAR': 10566})

### Random Oversampling

In [15]:
# Resample the training data with the RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({'QSO': 10566, 'STAR': 10566})

### Random Forest Classifier Algorithm (Galaxies vs. QSO) - Oversampling

In [16]:
# Train the Random Forest Classifier Model using the resampled data
model = RandomForestClassifier(n_estimators=100)
model.fit(X_resampled, y_resampled)

RandomForestClassifier()

In [17]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9998352010547132

In [18]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test,y_pred))

                   pre       rec       spe        f1       geo       iba       sup

        QSO       1.00      1.00      1.00      1.00      1.00      1.00      3034
       STAR       1.00      1.00      1.00      1.00      1.00      1.00      3523

avg / total       1.00      1.00      1.00      1.00      1.00      1.00      6557



In [19]:
# Display the confusion matrix
c= confusion_matrix(y_test,y_pred)
cmros = pd.DataFrame(c, index=["Actual - QSO", "Actual - Star"], columns=["Predicted - QSO","Predicted - Star"])
cmros

,Predicted - Galaxy,Predicted - Star
Actual - Galaxy,3033,1
Actual - Star,0,3523


### Random Undersampling

In [20]:
# Resample the training data with the RandomUnderSampler
rus = RandomUnderSampler(random_state=1)
X_resampled1, y_resampled1 = rus.fit_resample(X_train, y_train)
Counter(y_resampled1)

Counter({'QSO': 9103, 'STAR': 9103})

### Random Forest Classifier Algorithm (Star vs. QSO) - Undersampling

In [21]:
# Train the Random Forest Classifier Model using the resampled data
model = RandomForestClassifier(n_estimators=100)
model.fit(X_resampled1, y_resampled1)

RandomForestClassifier()

In [22]:
# Calculated the balanced accuracy score
y_pred2 = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred2)

0.9998352010547132

In [23]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test,y_pred2))

                   pre       rec       spe        f1       geo       iba       sup

        QSO       1.00      1.00      1.00      1.00      1.00      1.00      3034
       STAR       1.00      1.00      1.00      1.00      1.00      1.00      3523

avg / total       1.00      1.00      1.00      1.00      1.00      1.00      6557



In [26]:
# Display the confusion matrix
c= confusion_matrix(y_test,y_pred2)
cmrus = pd.DataFrame(c, index=["Actual - QSO","Actual - Star"], columns=["Predicted - QSO","Predicted - Star"])
cmrus

,Predicted - QSO,Predicted - Star
Actual - QSO,3033,1
Actual - Star,0,3523
